In [1]:
import logging
import pandas as pd
import glob
import math
from matplotlib import pyplot as plt
import numpy as np
import csv
import time
import gc
import scipy.stats
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import iqr
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from scipy.stats import entropy
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from pandas import read_csv
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
#import pywt
from scipy import signal
import matplotlib.pyplot as plt
from scipy import special
from pathlib import Path
import re
import os
from basys4ipps_ifw_agent.agent.extract_features import convert_sensor_data_to_tsfresh_format

# Prepare training data

In [5]:
from examples.test_data_generator import example_sensor_data, get_test_files

all_files = get_test_files("./train" )

training_index = list(range(20))

x_train, x_test = example_sensor_data(all_files, training_index, test_index=115, sensor=5)

Found 14 files!


c:\git\basys4ipps_ifw_agent\examples\test_data_generator.py:49: DtypeWarning: Columns (7,9,13,23,26,32,37,47,50,55,83,87,91,105,108,117,127,129,138,149,151,155,165,168,174,179,189,192,197,225,229,233,247,250,259,269,271,280,291,293,297,307,310,316,321,331,334,339,367,371,375,389,392,401,411,413,422,433,435,439,449,452,458,463,473,476,481,509,513,517,531,534,543,553,555,564,575,577,581,591,594,600,605,615,618,623,651,655,659,673,676,685,695,697,706,717,719,723,733,736,742,747,757,760,765,793,797,801,815,818,827,837,839,848,859,861,865,875,878,884,889,899,902,907,935,939,943,957,960,969,979,981,990,1001,1003,1007,1017,1020,1026,1031,1041,1044,1049,1077,1081,1085,1099,1102,1111,1121,1123,1132,1143,1145,1149,1159,1162,1168,1173,1183,1186,1191,1219,1223,1227,1241,1244,1253,1263,1265,1274,1285,1287,1291,1301,1304,1310,1315,1325,1328,1333,1361,1365,1369,1383,1386,1395,1405,1407,1416,1427,1429,1433,1443,1446,1452,1457,1467,1470,1475,1503,1507,1511,1525,1528,1537,1547,1549,1558,1569,1571,1575,1

# Train & Test

## Create configuration for training & testing

In [6]:
from basys4ipps_ifw_agent.agent.basys_agent import BasysAgent, BasysConfig
from pathlib import Path

basys_config = BasysConfig.load(create_if_none=True)
basys_config.alpha_safety_factor = 0.999
basys_config.tsfresh_features = Path(*Path(basys_config.tsfresh_features).parts[1:]).as_posix()
basys_config.tsfresh_random_forest = Path(*Path(basys_config.tsfresh_random_forest).parts[1:]).as_posix()
basys_config.use_tsfresh_features = True

Reading config file 'C:/git/BaSys4IPPS_IFW_Agent/examples/config.yaml'


In [7]:
# Ende Daten laden ######################################

# initialise model
# baseline_model = unified_outlier_score(base_model = "KNN", outlier_score_scaling = "gaussian", alpha_safety_factor = 0.99999, feature_group = "general_purpose", feature_scaling_method = "standardize", learning_type = "static", machine_component = "axis_drive", segementation_start = 20, segmentation_end = 710)

# fit the model on train data
# baseline_model.fit(X_train)

# predict using baseline model
# score_prob, alarm, exp_downtime = baseline_model.predict(X_test)

agent = BasysAgent(basys_config)

logging.info("Start training")
agent.fit(x_train, basys_config)

logging.info("Start testing")
score_prob, alarm, exp_downtime = agent.predict(x_test, basys_config)


INFO:BasysLogger:Using default StandardScaler
INFO:BasysLogger:Using KNN outlier detection n_neighbors=5, method=largest
INFO:BasysLogger:Extract features from training data
Feature Extraction: 100%|██████████| 20/20 [00:03<00:00,  5.92it/s]
INFO:BasysLogger:Fit the outlier detection model
INFO:BasysLogger:Extract features from test data
Feature Extraction: 100%|██████████| 1/1 [00:02<00:00,  2.96s/it]
INFO:BasysLogger:Execute decision function
INFO:BasysLogger:safety_factor=0.999000,
score_prob=[0.]
alarm=[False]
exp_downtime=1
